**IMPORT LIBRARIES AND CONSTANTS**

In [ ]:
%%capture

!pip install git-repo
!pip install wandb

import os
import sys
import shutil
import git
import torch
import wandb
from google.colab import drive
from google.colab import files

PROJECT_FOLDER_REL_PATH = "CNN OBJECT DETECTION"

drive.mount("/content/drive")
PROJECT_FOLDER_ABS_PATH = "/content/drive/My Drive/" + PROJECT_FOLDER_REL_PATH
sys.path.append(PROJECT_FOLDER_ABS_PATH)

from OD_constants import Constants

const = Constants(PROJECT_FOLDER_ABS_PATH)

**UNZIP ZIPPED SELECTED LABELED SECTIONS**

In [ ]:
ZIPPED_SELECTED_LABELED_SECTIONS_FILE_NAME = (const.LABELED_SECTIONS_FOLDER_REL_PATH + " - " + const.SELECTED_DATASET_NAME + " dataset").replace(" ", "_")
ZIPPED_SELECTED_LABELED_SECTIONS_FILE_ABS_PATH = PROJECT_FOLDER_ABS_PATH + "/" + ZIPPED_SELECTED_LABELED_SECTIONS_FILE_NAME

# if os.path.exists(ZIPPED_SELECTED_LABELED_SECTIONS_FILE_ABS_PATH + ".zip"):
#   if os.path.exists(const.SELECTED_LABELED_SECTIONS_FOLDER_ABS_PATH):
#     shutil.rmtree(const.SELECTED_LABELED_SECTIONS_FOLDER_ABS_PATH)
#   os.makedirs(const.SELECTED_LABELED_SECTIONS_FOLDER_ABS_PATH)

#   shutil.unpack_archive(ZIPPED_SELECTED_LABELED_SECTIONS_FILE_ABS_PATH + ".zip", const.SELECTED_LABELED_SECTIONS_FOLDER_ABS_PATH, "zip")
#   # os.remove(ZIPPED_SELECTED_LABELED_SECTIONS_FILE_ABS_PATH + ".zip")

**SETUP YOLO MODELS**

In [ ]:
# set the GPU hardware accelerator: "Runtime" -> "Change runtime type" -> "Hardware accelerator" -> "GPU"

print([torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else "CPU"])

if not os.path.exists(const.YOLO_MODELS_FOLDER_ABS_PATH):
  git.Repo.clone_from(const.YOLO_MODELS_URL, const.YOLO_MODELS_FOLDER_ABS_PATH)

%cd {const.YOLO_MODELS_FOLDER_ABS_PATH}
%pip install -qr requirements.txt

**DEFINE SELECTED DATASET CONFIG FILE**

In [ ]:
if not os.path.exists(const.SELECTED_DATASET_CONFIG_FOLDER_ABS_PATH):
    os.makedirs(const.SELECTED_DATASET_CONFIG_FOLDER_ABS_PATH)

with open(const.SELECTED_DATASET_CONFIG_FILE_ABS_PATH, "w") as f:
    f.write("train: {}\n".format(const.SELECTED_TRAINING_SECTION_IMAGES_FOLDER_ABS_PATH))
    f.write("test: {}\n".format(const.SELECTED_TESTING_SECTION_IMAGES_FOLDER_ABS_PATH))
    f.write("val: {}\n\n".format(const.SELECTED_VALIDATION_SECTION_IMAGES_FOLDER_ABS_PATH))
    f.write("nc: {}\n".format(len(const.SELECTED_GENUINE_CLASS_NAMES_LIST)))
    f.write("names: {}".format(const.SELECTED_GENUINE_CLASS_NAMES_LIST))

**DEFINE SELECTED DATASET HYPERPARAMETERS FILE**

In [ ]:
# CHANGEABLE VARIABLES:

FINAL_LEARNING_RATE = 0.005
INITIAL_LEARNING_RATE = FINAL_LEARNING_RATE / 2
MOMENTUM_RATE = 0.937
WEIGHT_DECAY = 0.0005
NUM_OF_WARMUP_EPOCHS = 10.0
WARMUP_MOMENTUM = 0.8
WARMUP_BIAS_LEARNING_RATE = 0.1

# # #


if not os.path.exists(const.SELECTED_DATASET_HYPERPARAMETERS_FOLDER_ABS_PATH):
    os.makedirs(const.SELECTED_DATASET_HYPERPARAMETERS_FOLDER_ABS_PATH)

with open(const.SELECTED_DATASET_HYPERPARAMETERS_FILE_ABS_PATH, "w") as f:
    f.write("lr0: {}\n".format(INITIAL_LEARNING_RATE))
    f.write("lrf: {}\n".format(FINAL_LEARNING_RATE))
    f.write("momentum: {}\n\n".format(MOMENTUM_RATE))
    f.write("weight_decay: {}\n".format(WEIGHT_DECAY))
    f.write("warmup_epochs: {}\n".format(NUM_OF_WARMUP_EPOCHS))
    f.write("warmup_momentum: {}\n".format(WARMUP_MOMENTUM))
    f.write("warmup_bias_lr: {}\n".format(WARMUP_BIAS_LEARNING_RATE))
    f.write("box: 0.05\n")
    f.write("cls: 0.5\n")
    f.write("cls_pw: 1.0\n")
    f.write("obj: 1.0\n")
    f.write("obj_pw: 1.0\n")
    f.write("iou_t: 0.2\n")
    f.write("anchor_t: 4.0\n")
    f.write("fl_gamma: 0.0\n")
    f.write("hsv_h: 0.015\n")
    f.write("hsv_s: 0.7\n")
    f.write("hsv_v: 0.4\n")
    f.write("degrees: 0.0\n")
    f.write("translate: 0.1\n")
    f.write("scale: 0.0\n")
    f.write("shear: 0.0\n")
    f.write("perspective: 0.0\n")
    f.write("flipud: 0.5\n")
    f.write("fliplr: 0.5\n")
    f.write("mosaic: 0.5\n")
    f.write("mixup: 0.0\n")
    f.write("copy_paste: 0.0")

**VISUALIZE TRAINING OF SELECTED YOLO MODEL**

In [ ]:
WANDB_API_KEY = "17f9c15e6adea63966157f01c6bd582f705f8f10"

wandb.login(key=WANDB_API_KEY)

# training of the selected YOLO model can be visualized at "https://wandb.ai/home"

**TRAIN SELECTED YOLO MODEL**

In [ ]:
# CHANGEABLE VARIABLES:

IMAGE_DIMENSION_SIZE = 640
SELECTED_YOLO_MODEL_CONFIG_FILE_NAME = "yolov5s.yaml"
TRAINING_BATCH_SIZE = 64
NUM_OF_EPOCHS = 100
YOLO_MODEL_WEIGHTS_FROM_TRAINED_INSTEAD_OF_SELECTED_MODEL = False
SELECTED_YOLO_MODEL_WEIGHTS_FILE_NAME = "yolov5s.pt"
TRAINED_YOLO_MODEL_FOLDER_NAME = "household_items_model_640DIM_0,005LR_100EP"  # trained YOLO model folder name is used here only if training starts with it's weights 
RESUME_TRAINING_PARTIALLY_TRAINED_YOLO_MODEL = False

# # #


if YOLO_MODEL_WEIGHTS_FROM_TRAINED_INSTEAD_OF_SELECTED_MODEL == True:
  TRAINED_YOLO_MODEL_WEIGHTS_FILE_NAME = "runs/train/" + TRAINED_YOLO_MODEL_FOLDER_NAME + "/weights/best.pt"
  SELECTED_OR_TRAINED_YOLO_MODEL_WEIGHTS_FILE_NAME = TRAINED_YOLO_MODEL_WEIGHTS_FILE_NAME
else:
  SELECTED_OR_TRAINED_YOLO_MODEL_WEIGHTS_FILE_NAME = SELECTED_YOLO_MODEL_WEIGHTS_FILE_NAME

DETAILED_SELECTED_DATASET_MODEL_NAME = const.SELECTED_DATASET_MODEL_NAME + "_" + str(IMAGE_DIMENSION_SIZE) + "DIM" + "_" + \
  str(FINAL_LEARNING_RATE).replace(".", ",") + "LR" + "_" + str(NUM_OF_EPOCHS) + "EP"

if RESUME_TRAINING_PARTIALLY_TRAINED_YOLO_MODEL == True:
  %cd {const.YOLO_MODELS_FOLDER_ABS_PATH}
  !python train.py --img {IMAGE_DIMENSION_SIZE} --cfg {SELECTED_YOLO_MODEL_CONFIG_FILE_NAME} --hyp \
    {const.SELECTED_DATASET_HYPERPARAMETERS_FILE_NAME} --batch {TRAINING_BATCH_SIZE} --epochs {NUM_OF_EPOCHS} --data \
    {const.SELECTED_DATASET_CONFIG_FILE_NAME} --weights {SELECTED_OR_TRAINED_YOLO_MODEL_WEIGHTS_FILE_NAME} --name \
    {DETAILED_SELECTED_DATASET_MODEL_NAME} --cache --resume
else:
  %cd {const.YOLO_MODELS_FOLDER_ABS_PATH}
  !python train.py --img {IMAGE_DIMENSION_SIZE} --cfg {SELECTED_YOLO_MODEL_CONFIG_FILE_NAME} --hyp \
    {const.SELECTED_DATASET_HYPERPARAMETERS_FILE_NAME} --batch {TRAINING_BATCH_SIZE} --epochs {NUM_OF_EPOCHS} --data \
    {const.SELECTED_DATASET_CONFIG_FILE_NAME} --weights {SELECTED_OR_TRAINED_YOLO_MODEL_WEIGHTS_FILE_NAME} --name \
    {DETAILED_SELECTED_DATASET_MODEL_NAME} --cache

**ZIP TRAINED YOLO MODEL FOLDER, COPY IT FOR SAVING ON LOCAL PC AND DOWNLOAD**

In [ ]:
# trained YOLO model folder name have to be changed for every new trained model

# CHANGEABLE VARIABLES:

TRAINED_YOLO_MODEL_FOLDER_NAME = "household_items_model_640DIM_0,005LR_100EP"

# # #


TRAINED_YOLO_MODEL_FOLDER_REL_PATH = "runs/train/" + TRAINED_YOLO_MODEL_FOLDER_NAME
TRAINED_YOLO_MODEL_FOLDER_ABS_PATH = const.YOLO_MODELS_FOLDER_ABS_PATH + "/" + TRAINED_YOLO_MODEL_FOLDER_REL_PATH

ZIPPED_TRAINED_YOLO_MODEL_FILE_NAME = TRAINED_YOLO_MODEL_FOLDER_NAME
ZIPPED_TRAINED_YOLO_MODEL_FILE_ABS_PATH = const.LOCAL_PC_FILES_FOLDER_ABS_PATH + "/" + ZIPPED_TRAINED_YOLO_MODEL_FILE_NAME

if not os.path.exists(const.LOCAL_PC_FILES_FOLDER_ABS_PATH):
  os.makedirs(const.LOCAL_PC_FILES_FOLDER_ABS_PATH)

shutil.make_archive(ZIPPED_TRAINED_YOLO_MODEL_FILE_ABS_PATH, "zip", TRAINED_YOLO_MODEL_FOLDER_ABS_PATH)

files.download(ZIPPED_TRAINED_YOLO_MODEL_FILE_ABS_PATH + ".zip")